In [1]:
import pandas as pd

In [2]:
file_path = './data/Individual_Provider_Network_Data__2021_Quarter_1.csv'
df_2021 = pd.read_csv(file_path)

C:\Users\kyung\AppData\Local\Temp\ipykernel_31036\3141834340.py:2: DtypeWarning: Columns (3,58,60,61,62,63,64,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv(file_path)


In [76]:
# Get the columns to keep (every other column)
selected_columns = ['Medicaid Provider Indicator', 'Borough/County Code', 'County', 'State']
df_filter_2021 = df_2021[selected_columns]
df_filter_2021 = df_filter_2021[df_filter_2021['State'] == 'NY']
df_filter_2021['Borough/County Code'] = df_filter_2021['Borough/County Code'].astype(int)

# Filter rows where 'County' is not equal to 0, extract the last three characters
df_filter_2021['Borough/County Code'] = df_filter_2021['Borough/County Code'].astype(str).str[-3:]

In [80]:
# Rename column names
df_filter_2021 = df_filter_2021.rename(columns={df_filter_2021.columns[1]: 'COUNTYFP'})

# Group by 'county' column and count occurrences of 0 or 1 in 'indicator' column
grouped = df_filter_2021.groupby(['COUNTYFP'])['Medicaid Provider Indicator'].value_counts().unstack().fillna(0)

# Calculate ratio of 1s to the total occurrences of 0s and 1s
grouped['medicaid_acceptance_rate'] = grouped[1] / (grouped[0] + grouped[1])
grouped['medicaid_acceptance_rate'] = (grouped['medicaid_acceptance_rate'] * 100).round(1)

# Reset index to keep 'COUNTYFP' as a column
grouped = grouped.reset_index()

# Rename column names
grouped = grouped.rename(columns={grouped.columns[1]: 'medicaid_rejected'})
grouped = grouped.rename(columns={grouped.columns[2]: 'medicaid_accepted'})

In [81]:
# File path where you want to save the CSV file
output_file_path = './data/outputs/medicaid_acceptance_rate_counties_2021_Quater1.csv'

# Export 'result_df' DataFrame to a CSV file
grouped.to_csv(output_file_path, index=False)